In [7]:
import pandas as pd
import numpy as np

# Read the CSV file
data = pd.read_csv('c4_game_database.csv')
data.head()

# Select columns pos01 to pos42
columns = [f'pos_{i:02d}' for i in range(1, 43)]
selected_data = data[columns]

# Function to convert each row to a 6x7 2D array
def row_to_2d_array(row):
    return np.array(row).reshape(6, 7)

# Apply the function to each row
data_2d_arrays = selected_data.apply(row_to_2d_array, axis=1)

# Display the first 2D array to verify
print(data_2d_arrays.iloc[0])

[[ 1.  1.  1. -1. -1.  1.  0.]
 [-1. -1. -1.  1. -1. -1.  0.]
 [-1.  1.  1. -1.  1.  1.  0.]
 [ 1. -1.  1. -1. -1. -1. -1.]
 [ 1.  1. -1. -1.  1.  1.  1.]
 [-1.  1. -1.  1. -1.  1. -1.]]
